In [13]:
import numpy as np
import face_detection
import os, python_speech_features
import scipy.io.wavfile as wav
import pandas as pd
import torch.nn.functional as F
from talkNet import talkNet
from dataset import MyDataset
from torch.utils.data import DataLoader
import tqdm
import torch
import cv2
import tools
import subprocess
from collections import defaultdict
import whisper
detector = face_detection.build_detector(
"DSFDDetector", confidence_threshold=.3, nms_iou_threshold=.5) #DSFDDetector

c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
inputVideo = "C:/Users/jmmol/Desktop/LIP-RTVE/MP4s/speaker270/speaker270_0002.mp4"
#inputVideo = r"inputs/dos_personas_hablando_por_turnos.mp4"
videoDuration = tools.checkVideoDuration(inputVideo)
videoFrames = int(videoDuration*25)
#tools.splitVideo(inputVideo,2,videoDuration)

In [3]:
#Carga vídeo
res, facePos, faceFrames = tools.saveMultiFace(inputVideo,detector,50)
# AUDIO PROCESSING
audioPath = tools.convert_video_to_audio_ffmpeg(inputVideo)
_,sig = wav.read(audioPath)#r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker042_0063.wav") #
# _,sig = wav.read(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav") 
# audioPath = r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav"
audio = python_speech_features.mfcc(sig, 16000, numcep = 13, winlen = 0.025, winstep = 0.010) #ASUME VIDEO A 25 Y AUDIO A 100, MODIFICAR
print("Nº CARAS:",len(res))
for cara in res.keys():
    print(len(res[cara]))


Nº CARAS: 5
1
1
245
241
238


In [4]:
print(faceFrames[2])
# cv2.imshow("My Video", res[3][50])
# cv2.waitKey(0)

[0, 1, 1, 2, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 

In [5]:
model = talkNet()
model.load_state_dict(torch.load("./exps/exp1/model/model21_0006.model"))
windowSize = 21

06-14 13:55:28 Model para number = 15.01


In [6]:
videoDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/npz"
audioDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/mfccs"
datasetTrain = MyDataset(51,videoDir,audioDir,"testSamples.csv")
item = datasetTrain.__getitem__(4309)
valLoader = DataLoader(dataset=datasetTrain,shuffle=False,batch_size=32,num_workers=14) #Cambiar num_workers

25 100


In [7]:
# correctFrames, totalFrames = 0, 0
# correctSamples, totalSamples = 0, 0
# totalPreds = []
# for num, (audioFeature, visualFeature, labels) in enumerate(tqdm.tqdm(valLoader)):
#     with torch.no_grad():    
#         output = model((audioFeature,visualFeature))
#         labels = labels.cuda()
#         batchPreds = torch.reshape(output[1], labels.shape)
#         #Precision a nivel de video
#         videoPreds = torch.mode(batchPreds,dim=1)[0]
#         totalPreds.extend(videoPreds.detach().cpu().numpy().astype(int))
#         labelMode = torch.mode(labels,dim=1)[0]
#         correctSamples += (videoPreds == labelMode).sum().float()
#         totalSamples += len(labels)
#         # Precision a nivel de frame
#         labels = labels.reshape((-1))
#         correctFrames += (output[1] == labels).sum().float()
#         totalFrames += len(labels) 
# #7865
# print("Frame acc:", correctFrames/totalFrames)
# print("Sample acc:", correctSamples/totalSamples)
center = 47
iAudio = tools.padAudio(audio,1,center,51,len(res[0]))
iVideo = tools.padVideo(res[0],center,51)
print(iAudio[40],item[0][40])
print("Audios iguales?",(iAudio == item[0]).sum(),iAudio.shape[0]*iAudio.shape[1])
print("Videos iguales?",(iVideo == item[1]).sum(),iVideo.shape[0]*112*112)
print(audio[0],item[0][0])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([ 16.0407,   5.1092,  -7.4174,  26.3697, -21.4168, -34.9214, -26.8578,
        -27.1986, -11.6547,  -2.2427, -25.6342, -12.2112,   4.5576])
Audios iguales? tensor(0) 2652
Videos iguales? tensor(4) 639744
[ 20.70734516 -30.64465945  24.06522304 -17.63905206  14.91974731
 -13.65559986  -3.76110274   6.57601747  -6.36454786  -1.20602679
 -15.86244925  -2.55367867   5.93990086] tensor([ 11.6267,   6.2778,   2.7260,  -5.3083,  -9.2899,  -7.2238,  -8.3037,
         -3.4917,  -6.4616, -10.6301, -17.5955,  -8.8042,  -3.2956])


c:\Users\jmmol\Desktop\COSAS V7\TFM\tools.py:182: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  video = torch.FloatTensor(video)


In [8]:
predArray = defaultdict(list)
# MEDIA FRAMES
for actualSpeaker in res.keys():
    print("SPEAKER", actualSpeaker)
    for i in range(len(res[actualSpeaker])):
        #center = i
        center = faceFrames[actualSpeaker][i]
        print("FRAME Nº ",center)
        iAudio = tools.padAudio(audio,1,center,windowSize,videoFrames).unsqueeze(0)
        iVideo = tools.padVideo(res[actualSpeaker],center-faceFrames[actualSpeaker][0],windowSize).unsqueeze(0)
        #print("audio shape: ", audio.shape)
        #print("video shape: ", video.shape)
        # iAudio = item[0].unsqueeze(0)
        # iVideo = item[1].unsqueeze(0)
        print(iAudio[0][0][0:5],iVideo[0][0][0][0:5])
        output = model((iAudio,iVideo))
        #output = model((audio,video))
        #print(output)
        print("Pred ->",torch.mode(output[1])[0],"\tProb Pos ->",torch.mean(output[1]))
        predArray[actualSpeaker].append(torch.mean(output[1]).detach().cpu().numpy())

# SECUENCIAL
# for actualSpeaker in res.keys():
#     print("SPEAKER", actualSpeaker)
#     for i in range(0,len(res[actualSpeaker]),windowSize):
#         #center = i
#         center = faceFrames[actualSpeaker][i]
#         #print("FRAME Nº ",center)
#         iAudio = tools.padAudio(audio,1,center,windowSize,videoFrames).unsqueeze(0)
#         iVideo = tools.padVideo(res[actualSpeaker],center-faceFrames[actualSpeaker][0],windowSize).unsqueeze(0)
#         #print(iAudio[0][0][0:5],iVideo[0][0][0][0:5])
#         output = model((iAudio,iVideo))
#         predArray[actualSpeaker].extend(output[1].detach().cpu().numpy().tolist())


SPEAKER 0
FRAME Nº  0
tensor([0., 0., 0., 0., 0.]) tensor([0., 0., 0., 0., 0.])
Pred -> tensor(0., device='cuda:0') 	Prob Pos -> tensor(0., device='cuda:0')
SPEAKER 1
FRAME Nº  0
tensor([0., 0., 0., 0., 0.]) tensor([0., 0., 0., 0., 0.])
Pred -> tensor(0., device='cuda:0') 	Prob Pos -> tensor(0., device='cuda:0')
SPEAKER 2
FRAME Nº  0
tensor([0., 0., 0., 0., 0.]) tensor([0., 0., 0., 0., 0.])
Pred -> tensor(0., device='cuda:0') 	Prob Pos -> tensor(0., device='cuda:0')
FRAME Nº  1
tensor([0., 0., 0., 0., 0.]) tensor([0., 0., 0., 0., 0.])
Pred -> tensor(0., device='cuda:0') 	Prob Pos -> tensor(0., device='cuda:0')
FRAME Nº  1
tensor([0., 0., 0., 0., 0.]) tensor([0., 0., 0., 0., 0.])
Pred -> tensor(0., device='cuda:0') 	Prob Pos -> tensor(0., device='cuda:0')
FRAME Nº  2
tensor([0., 0., 0., 0., 0.]) tensor([0., 0., 0., 0., 0.])
Pred -> tensor(0., device='cuda:0') 	Prob Pos -> tensor(0.1905, device='cuda:0')
FRAME Nº  2
tensor([0., 0., 0., 0., 0.]) tensor([0., 0., 0., 0., 0.])
Pred -> tensor

In [18]:
def createVideo(videoName,imgFrames,audioPath,width,height):
    video = cv2.VideoWriter("output.mp4", 0, 25, (width,height))

    for image in imgFrames:
        video.write(image)

    cv2.destroyAllWindows()
    video.release()
    print(' '.join(["ffmpeg","-y","-i",os.getcwd()+"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+"/output2.mp4"]))
    res = subprocess.check_output(["ffmpeg","-y","-i",os.getcwd()+f"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+f"/outputs/videos/{videoName}"])
    os.remove(os.getcwd()+f"/output.mp4")
    
    print(res)

In [19]:
#Asignación de bounding boxes y probabilidades al video de output
os.makedirs("outputs/videos", exist_ok=True)
cap = cv2.VideoCapture(inputVideo)
videoImages = []
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True:
    ret, image = cap.read()
    videoImages.append(image)
    if ret == False:
        break


for speakerN in predArray.keys():
    for i, fr in enumerate(faceFrames[speakerN]): #Para cada frame en el que aparezca la cara
        image = videoImages[fr]
        greenValue = int(255*predArray[speakerN][i])
        redValue = 255-greenValue
        color = (0,greenValue,redValue)
        #print(color)
        print(speakerN,i)
        xmin,ymin,xmax,ymax = facePos[speakerN][i]
        image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax),color , 1)
        cv2.putText(image, str(predArray[speakerN][i]), (xmin, ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1)

        # cv2.imshow("My Video", image)
        # cv2.waitKey(0)


cap.release()
cv2.destroyAllWindows()
print(len(videoImages))
path = os.path.normpath(inputVideo)
videoName = str(path.split(os.sep)[-1])
createVideo(videoName,videoImages,audioPath,width,height)
#print(predArray[3])

0 0
1 0
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
2 33
2 34
2 35
2 36
2 37
2 38
2 39
2 40
2 41
2 42
2 43
2 44
2 45
2 46
2 47
2 48
2 49
2 50
2 51
2 52
2 53
2 54
2 55
2 56
2 57
2 58
2 59
2 60
2 61
2 62
2 63
2 64
2 65
2 66
2 67
2 68
2 69
2 70
2 71
2 72
2 73
2 74
2 75
2 76
2 77
2 78
2 79
2 80
2 81
2 82
2 83
2 84
2 85
2 86
2 87
2 88
2 89
2 90
2 91
2 92
2 93
2 94
2 95
2 96
2 97
2 98
2 99
2 100
2 101
2 102
2 103
2 104
2 105
2 106
2 107
2 108
2 109
2 110
2 111
2 112
2 113
2 114
2 115
2 116
2 117
2 118
2 119
2 120
2 121
2 122
2 123
2 124
2 125
2 126
2 127
2 128
2 129
2 130
2 131
2 132
2 133
2 134
2 135
2 136
2 137
2 138
2 139
2 140
2 141
2 142
2 143
2 144
2 145
2 146
2 147
2 148
2 149
2 150
2 151
2 152
2 153
2 154
2 155
2 156
2 157
2 158
2 159
2 160
2 161
2 162
2 163
2 164
2 165
2 166
2 167
2 168
2 169
2 170
2 171
2 172
2 173
2 174
2 175
2 176
2 177
2 178
2 179
2 180
2 181
2 182
2 18

In [21]:
# GUARDAR DATOS 
model = whisper.load_model("small")
transcription = model.transcribe(audioPath, language="es", verbose=True)
os.makedirs("outputs/npz", exist_ok=True)
np.savez_compressed(r"outputs/npz/"+videoName,facePos=facePos,faceFrames=faceFrames,preds=predArray,transcription=transcription)


[00:00.000 --> 00:05.280]  Hace 25 años la ONU declaró el 3 de diciembre como Día Internacional de las Personas con
[00:05.280 --> 00:09.440]  Discapacidad, con el objetivo de concienciarnos a todos sobre su situación y, en su caso,
[00:09.440 --> 00:09.440]  el 3 de diciembre.


In [12]:
#createVideo(res[3],audioPath,112,112)

[00:00.000 --> 00:05.280]  Hace 25 años la ONU declaró el 3 de diciembre como Día Internacional de las Personas con
[00:05.280 --> 00:09.440]  Discapacidad, con el objetivo de concienciarnos a todos sobre su situación y, en su caso,
[00:09.440 --> 00:09.440]  el 3 de diciembre.


In [26]:
loaded = np.load(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\outputs\npz\speaker270_0002.mp4.npz",allow_pickle=True)

In [27]:
loaded["transcription"]

array({'text': ' Hace 25 años la ONU declaró el 3 de diciembre como Día Internacional de las Personas con Discapacidad, con el objetivo de concienciarnos a todos sobre su situación y, en su caso,', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 5.28, 'text': ' Hace 25 años la ONU declaró el 3 de diciembre como Día Internacional de las Personas con', 'tokens': [50364, 389, 617, 3552, 11424, 635, 9299, 52, 16694, 812, 806, 805, 368, 14285, 49772, 2617, 413, 2686, 4844, 13608, 368, 2439, 8443, 296, 416, 50628], 'temperature': 0.0, 'avg_logprob': -0.5163383796566823, 'compression_ratio': 1.3311258278145695, 'no_speech_prob': 0.09169353544712067}, {'id': 1, 'seek': 0, 'start': 5.28, 'end': 9.44, 'text': ' Discapacidad, con el objetivo de concienciarnos a todos sobre su situación y, en su caso,', 'tokens': [50628, 4208, 9485, 326, 4580, 11, 416, 806, 29809, 368, 416, 537, 30322, 24979, 257, 6321, 5473, 459, 29343, 288, 11, 465, 459, 9666, 11, 50836], 'temperature': 0.0, 'avg_logprob'